# Collect Metrics from HADDOCK 3 Experiment Outputs

In [2]:
import os
import pandas as pd

In [10]:
## Read in Experiments worksheet
experiments = pd.read_excel('../../Experiments.xlsx', sheet_name='Experiments').head(0)
## For testing...
experiments = experiments.append({'experiment_id': 'TEST_5A3I'}, ignore_index=True)

# experiments.head()

,experiment_id,antibody_id,antigen_id,submitted,haddock_best_cluster,haddock_best_pdb_path,haddock_Nstruc,haddock_Evdw_plus_0.1Eelec,haddock_Evdw_plus_0.1Eelec_sd,haddock_Evdw,...,haddock_AIRviol,haddock_AIRviol_sd,haddock_dihedviol,haddock_dihedviol_sd,haddock_BSA,haddock_BSA_sd,haddock_score,haddock_score_sd,haddock_prodigy_deltaG_kcalpermol,haddock_prodigy_dissociation_constant_M
0,TEST_5A3I,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [27]:
## Create empty dataframe to store results
metrics_df = pd.DataFrame(columns=['experiment_id', 'cluster_rank', 'cluster_id', 'n', 'under_eval',
                                   'score', 'score_std', 'irmsd', 'irmsd_std',
                                   'fnat', 'fnat_std', 'lrmsd', 'lrmsd_std', 'dockq', 'dockq_std',
                                   'air', 'air_std', 'bsa', 'bsa_std',
                                   'desolv', 'desolv_std', 'elec', 'elec_std',
                                   'total', 'total_std', 'vdw', 'vdw_std',
                                   'caprieval_rank', 'best_pdb_path'])

# metrics_df

In [24]:
for index, experiment in experiments.iterrows():
    experiment_id = experiment['experiment_id']
    print(f"Getting metrics for experiment: {experiment_id}")

    ## Check if the output/ directory exists
    outputs_dir = f"../../data/experiments/{experiment_id}/output/analysis/6_caprieval_analysis"
    if os.path.exists(outputs_dir):
        ## Read in the cluster metrics .tsv file, skipping rows that start with # character
        cluster_metrics_file = f"{outputs_dir}/capri_clt.tsv"
        metrics_df_iter = pd.read_csv(cluster_metrics_file, sep='\t', comment='#')
        ## Filter to the row with the lowest vdw score
        metrics_df_iter_best = metrics_df_iter[metrics_df_iter['vdw'] == metrics_df_iter['vdw'].min()]
        best_cluster = metrics_df_iter_best.iloc[0]['cluster_id']
        
        ## Read in the individual metrics file
        ss_metrics_file = f"{outputs_dir}/capri_ss.tsv"
        ss_df_iter = pd.read_csv(ss_metrics_file, sep='\t', comment='#')
        ## Filter to the rows with the best cluster
        ss_df_iter = ss_df_iter[ss_df_iter['cluster-id'] == best_cluster]
        ## Find the best PDB with the lowest vdw score
        ss_df_iter_best = ss_df_iter[ss_df_iter['vdw'] == ss_df_iter['vdw'].min()].iloc[0]

        ## Add the best PDB path to the metrics dataframe
        metrics_df_iter_best['best_pdb_path'] = ss_df_iter_best['model']
        metrics_df_iter_best['experiment_id'] = experiment_id
        metrics_df = metrics_df.append(metrics_df_iter_best)

    else:
        print(f"\tNo output/ directory found.")
        continue


Getting metrics for experiment: TEST_5A3I


In [25]:
metrics_df

,experiment_id,cluster_rank,cluster_id,n,under_eval,score,score_std,irmsd,irmsd_std,fnat,...,desolv,desolv_std,elec,elec_std,total,total_std,vdw,vdw_std,caprieval_rank,best_pdb_path
0,TEST_5A3I,-,-,10,-,15.923,6.469,8.819,7.485,0.369,...,3.63,3.176,-284.209,31.112,1065.848,91.206,-73.191,6.087,1,../../5_mdref/mdref_1.pdb
0,TEST_5A3I,-,-,10,-,15.923,6.469,8.819,7.485,0.369,...,3.63,3.176,-284.209,31.112,1065.848,91.206,-73.191,6.087,1,../../5_mdref/mdref_1.pdb
